In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [2]:
df = pd.read_csv('Dataset.csv')

In [3]:
df.drop(['Loan Identifier','EMI','UPB_diff','Interest_Component','PREPAYMENT_AMT','Remaining Months To Maturity'], axis=1, inplace = True)

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=df.drop(['Current Actual UPB'],axis=1)
y = df['Current Actual UPB']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [5]:
X_train

,Monthly Reporting Period,Channel,Original Interest Rate,Current Interest Rate,Original UPB,Original Loan Term,Origination Date,First Payment Date,Original Loan to Value Ratio (LTV),Debt-To-Income (DTI),...,Modification Flag,Zero Balance Code,Special Eligibility Program,Property Valuation Method,High Balance Loan Indicator,Unemployment_rate,Divorce_rate,Inflation_rate,Previous_UPB,Remaining MFM
417042,52016,1,6.125,6.125,180000.0,360,12007,32007,85,29.0,...,0,0,0,4,0,4.9,3.0,1.0,154268.17,248
579241,22010,1,4.875,4.875,279000.0,360,92009,112009,74,22.0,...,0,0,0,4,0,9.6,3.6,2.1,279000.00,355
524775,72017,2,6.000,6.000,116000.0,360,92008,122008,80,22.0,...,0,0,0,4,0,4.4,2.9,1.7,100222.76,254
252455,32013,2,6.000,6.000,72000.0,360,12004,32004,58,9.0,...,0,0,0,4,0,7.4,3.3,1.5,60843.37,250
706737,22013,1,3.375,3.375,223000.0,180,102011,122011,70,23.0,...,0,0,0,4,0,7.4,3.3,2.0,207375.46,164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1255245,122020,2,2.990,2.990,200000.0,360,112020,12021,95,32.0,...,0,0,0,0,0,8.1,2.3,1.4,200000.00,359
732029,42022,2,4.000,4.000,270000.0,360,42012,62012,43,35.0,...,0,0,0,4,0,3.6,2.3,8.3,213869.59,240
140929,102008,0,7.000,7.000,162000.0,360,12002,32002,90,38.0,...,0,0,0,4,0,5.8,3.5,3.7,147584.79,279
635475,82014,1,5.375,5.375,46000.0,360,62010,82010,75,32.0,...,0,0,0,4,0,6.2,3.2,1.7,42700.72,310


In [6]:
y_train

417042     153961.88
579241     279000.00
524775     100030.19
252455      60587.81
706737     206365.05
             ...    
1255245    200000.00
732029     213293.46
140929     147260.97
635475      42631.41
113370     124000.00
Name: Current Actual UPB, Length: 1005174, dtype: float64

In [7]:
model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=X.shape[1]))

model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=16, activation='relu'))

model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_absolute_error')

In [8]:
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32) 

Epoch 1/10
31412/31412 [==============================] - 49s 2ms/step - loss: 5976.0967
Epoch 2/10
31412/31412 [==============================] - 83s 3ms/step - loss: 407.1357
Epoch 3/10
31412/31412 [==============================] - 52s 2ms/step - loss: 377.6118
Epoch 4/10
31412/31412 [==============================] - 93s 3ms/step - loss: 368.9288
Epoch 5/10
31412/31412 [==============================] - 91s 3ms/step - loss: 361.8377
Epoch 6/10
31412/31412 [==============================] - 61s 2ms/step - loss: 355.5240
Epoch 7/10
31412/31412 [==============================] - 78s 2ms/step - loss: 348.4727
Epoch 8/10
31412/31412 [==============================] - 90s 3ms/step - loss: 345.5592
Epoch 9/10
31412/31412 [==============================] - 91s 3ms/step - loss: 342.1930
Epoch 10/10
31412/31412 [==============================] - 102s 3ms/step - loss: 339.8487


In [9]:
predictions = model.predict(X_test_scaled)

7853/7853 [==============================] - 9s 1ms/step


In [10]:
print(predictions)

[[149726.66]
 [155314.75]
 [ 31823.85]
 ...
 [330110.3 ]
 [139788.92]
 [138923.5 ]]


In [11]:
print(y_test)

630107     149915.96
272561     155220.87
322664      31762.56
1120281         0.00
46070      111689.17
             ...    
453709      70475.48
731933     260716.44
387529     329924.70
1055537    140000.00
510489     138990.74
Name: Current Actual UPB, Length: 251294, dtype: float64


In [12]:
from sklearn.metrics import mean_absolute_error
mean_absolute_error(predictions, y_test)

315.6805563676618

In [13]:
from sklearn.metrics import r2_score
print(r2_score(predictions, y_test))

0.9993583366067434


In [ ]:
import pickle
file=open('dnn_a3.pkl','wb')
pickle.dump(model, file)